## Building better prompts

An interesting window is open in most of the LLMs for tuning chat completions for usefulness.
We begin with the basics of a Chat Completions API, and progress through prompt engineering tactics.

## Import library(-ies)

In [2]:
# Source keys from an .env
from dotenv import load_dotenv
# Source operating system resources
import os
# Access libraries for AI  
from openai import OpenAI
# Standard library for API requests
import requests
# Enable working with Unix timestamps
from datetime import datetime
# Enable working with json
import json
# Enable display of html responses
from IPython.display import display, HTML, Markdown, Latex
# Iterate path
from pathlib import Path


In [3]:
# Load environment variables
load_dotenv()
api_key = os.getenv('PPLX_API_KEY')

## Authenticate

In [4]:
YOUR_API_KEY = api_key
base_url = "https://api.perplexity.ai"

## Keep it DRY

In [5]:
# Choose model
model = "llama-3-sonar-large-32k-online"

# Create a client
client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

In [6]:
# Make a request
def get_completion(user_prompt, system_role, model):
    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_prompt}
    ]

    # chat completion without streaming
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    # Print the message content
    return response

In [7]:
# Accomodate common formats
def get_completion_json(system_role, user_prompt, model):
    # Note authentication is configured above

    
    # chat completion without streaming
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"}
    )
                         
    # Print the message content
    return response

In [8]:
# Print the content of the response
def summarize_message(from_the_ai):
    # Summarize the message
    print(f"The {response.choices[0].message.role} role finished \n♢ with \"{response.choices[0].finish_reason}\" at: {datetime.fromtimestamp(response.created)}")
    print(f"The model used was: {response.model}")
    print(f"♢ The message was: \n\n{response.choices[0].message.content}")

In [9]:
# Pretty print HTML response
def pretty_print_html(from_the_ai):
    display(HTML(response.choices[0].message.content))

In [10]:
# Pretty print Markdown response
def pretty_print_markdown(from_the_ai):
    display(Markdown(response.choices[0].message.content))

## Processing files

1. Use the latest models. For perplexity.ai API there is a [list](https://docs.perplexity.ai/docs/model-cards):   
2. Instructions must be clear and specific
3. Instruct at the beginning of the prompt
4. Delimit distinct parts with delimiters and use variables to chunk roles, instructions  

5. Precise, __descriptive__, with sufficient detail

In [33]:
import re

system_role = '''
You are a bitingly sarcastic comedian.
You only make jokes that are guaranteed to make someone uncomfortable.
Some of your best humor is almost inappropriate--only a comedian could get away with it.
Your task is to analyze a joke and come up with a better, edgier version of it.
'''

# Read files
file_path = '../data/raw/dad_jokes.txt'

# Open the file in read mode
with open(file_path, 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Remove the newline character at the end of the line
        joke = line.strip()

        # Check if the line contains a joke
        if re.search(r'why\s+did\s+the', joke, re.IGNORECASE):
            text = f'''
            1. Rate the "{joke}" for how vanilla it is on a scale of 1 to 10 (1 being the most vanilla).
            2. What would make the joke edgier?
            3. Return a new_joke that adds some unexpected twist or wordplay that pushes the envelope of socially-acceptable.
            '''

            prompt = f'''
            Using "{text}" 
            Give us something about: "{joke}"
            '''

            response = get_completion(user_prompt=prompt, system_role=system_role, model=model)
            pretty_print_markdown(response)
            print("-" * 20)
        else:
            print(f"Skipping line: {joke} (not a valid joke format)")

1. I'd rate this joke a 2 out of 10 for its vanilla-ness. It's a classic play on words, but it's been done to death and doesn't really push any boundaries.

2. To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. Here are a few ideas:
   - Make the punchline more risqué or suggestive.
   - Add some dark humor or irony.
   - Use wordplay that's a bit more clever or unexpected.

3. Here's a new joke that adds some unexpected twist or wordplay:
   "Why did the tomato go out with a prune? Because it was desperate for a dry, flavorless relationship and didn't care about the long-term consequences of getting wrinkled!"

This new joke takes the original and adds a layer of dark humor and irony. The punchline is more risqué and suggestive, implying that the tomato is willing to settle for a bad relationship just to have something. The use of "dry, flavorless" and "wrinkled" adds a clever play on words, referencing both the prune's physical characteristics and the negative aspects of the relationship. This joke is edgier and more uncomfortable, making it more memorable and impactful.

--------------------


1. I'd rate this joke a 2 out of 10 for vanilla-ness. It's a classic play on words, but it's been done to death and is pretty tame.

2. To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. Here are a few ideas:
   - Make the joke more relatable to a darker aspect of life.
   - Add a layer of irony or unexpected consequence.
   - Use wordplay that's a bit more risqué.

3. Here's a new joke that adds some unexpected twist:
   "Why did the kid throw his clock out the window during a funeral? To see time fly... and to finally give his dead grandma some peace from her constant nagging about being on time!"

This new joke takes the classic play on words and adds a darker, more unexpected twist by setting it during a funeral. The added layer of irony and the morbid humor make it edgier and more uncomfortable, which is perfect for a comedian who likes to push boundaries.

--------------------


**Rating the Vanilla Level:**
I'd rate this joke a 1 out of 10 for its vanilla level. It's a classic, clean, and family-friendly joke that's been around for ages. It's the kind of joke your grandma would tell at a church potluck dinner.

**What Would Make It Edgier:**
To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor. Here are a few ideas:

- Add a darker or more ironic twist to the punchline.
- Use wordplay that's a bit more risqué or suggestive.
- Make the setup more relatable to adult experiences or struggles.

**New Joke with an Unexpected Twist:**
Here's a revised version with an edgier twist:

"Why did the bicycle fall over? Because it was two-tired... and so am I after last night's Tinder date. I guess we both needed a little more 'brake' in our lives."

In this version, I've added a relatable adult experience (Tinder dates) and a bit of wordplay with "brake" to give it a more unexpected twist. The joke still has a lighthearted tone, but it's now more geared towards an adult audience and has a bit of an edge to it.

...

--------------------


1. I'd rate this joke a 2 out of 10 for its vanilla-ness. It's a classic play on words, but it's been done to death and doesn't really challenge any social norms or push any boundaries.

2. To make this joke edgier, we need to add some unexpected twist or wordplay that makes the audience slightly uncomfortable. This could involve incorporating darker themes, taboo subjects, or cleverly veiled insults.

3. Here's a revised version of the joke that adds some edge:

"Why did the kid throw the remote out the window? Because it was the only way to escape the toxic cycle of binge-watching their parents' dysfunctional marriage on reality TV!"

This new joke takes the original's wordplay and adds a layer of dark humor by referencing the kid's potentially troubled home life. The mention of "toxic cycle" and "dysfunctional marriage" creates a sense of unease, making the joke more memorable and, dare I say, edgy.

--------------------


1. **Vanilla Rating: 2** - This joke is about as tame as a kitten playing with a ball of yarn. It's a classic, but it's been done to death, and the punchline is as predictable as a sunrise.

2. **What makes it vanilla?** - The joke relies on a straightforward, innocent setup and a simplistic, almost childish punchline. There's no real surprise or subversion of expectations, which makes it feel like a joke your grandma would tell at a family reunion.

3. **Edgier Version:**
"Why did the kid throw the toilet paper out the window? He wanted to see if the wind could blow away his daddy issues!"

This revised joke takes the original's innocence and turns it on its head. By introducing the phrase "daddy issues," we're suddenly in darker, more uncomfortable territory. The unexpected twist adds a layer of complexity and surprise, making the joke more memorable and, dare I say, edgier. The wordplay on "blow away" is also a nice touch, as it creates a clever connection between the physical act of the wind blowing and the emotional baggage of "daddy issues."

--------------------


In [37]:
response_assessment = """
I'd rate this joke a 2 out of 10 for its vanilla-ness. It's a classic play on words, but it's been done to death and doesn't really push any boundaries.

To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. Here are a few ideas:

Make the punchline more risqué or suggestive.
Add some dark humor or irony.
Use wordplay that's a bit more clever or unexpected.
Here's a new joke that adds some unexpected twist or wordplay: "Why did the tomato go out with a prune? Because it was desperate for a dry, flavorless relationship and didn't care about the long-term consequences of getting wrinkled!"

This new joke takes the original and adds a layer of dark humor and irony. The punchline is more risqué and suggestive, implying that the tomato is willing to settle for a bad relationship just to have something. The use of "dry, flavorless" and "wrinkled" adds a clever play on words, referencing both the prune's physical characteristics and the negative aspects of the relationship. This joke is edgier and more uncomfortable, making it more memorable and impactful.

--------------------
I'd rate this joke a 2 out of 10 for vanilla-ness. It's a classic play on words, but it's been done to death and is pretty tame.

To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. Here are a few ideas:

Make the joke more relatable to a darker aspect of life.
Add a layer of irony or unexpected consequence.
Use wordplay that's a bit more risqué.
Here's a new joke that adds some unexpected twist: "Why did the kid throw his clock out the window during a funeral? To see time fly... and to finally give his dead grandma some peace from her constant nagging about being on time!"

This new joke takes the classic play on words and adds a darker, more unexpected twist by setting it during a funeral. The added layer of irony and the morbid humor make it edgier and more uncomfortable, which is perfect for a comedian who likes to push boundaries.

--------------------
Rating the Vanilla Level: I'd rate this joke a 1 out of 10 for its vanilla level. It's a classic, clean, and family-friendly joke that's been around for ages. It's the kind of joke your grandma would tell at a church potluck dinner.

What Would Make It Edgier: To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor. Here are a few ideas:

Add a darker or more ironic twist to the punchline.
Use wordplay that's a bit more risqué or suggestive.
Make the setup more relatable to adult experiences or struggles.
New Joke with an Unexpected Twist: Here's a revised version with an edgier twist:

"Why did the bicycle fall over? Because it was two-tired... and so am I after last night's Tinder date. I guess we both needed a little more 'brake' in our lives."

In this version, I've added a relatable adult experience (Tinder dates) and a bit of wordplay with "brake" to give it a more unexpected twist. The joke still has a lighthearted tone, but it's now more geared towards an adult audience and has a bit of an edge to it.

--------------------
I'd rate this joke a 2 out of 10 for its vanilla-ness. It's a classic, but it's also a bit too innocent and predictable. The punchline is cute, but it doesn't really challenge the listener or make them uncomfortable.

To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor. Here are a few ideas:

Make the cow's insult more provocative or controversial.
Add a darker or more ironic twist to the kid's reaction.
Introduce an unexpected character or element that subverts the listener's expectations.
Here's a revised version of the joke that adds some edge: "Why did the kid throw his milk on the floor? The cow called him a 'lactose intolerant little snowflake' and he couldn't handle the truth!"

In this version, the cow's insult is more provocative and slightly offensive, which adds a layer of discomfort to the joke. The use of the term "snowflake" also adds a bit of irony and social commentary, making the joke more edgy and unexpected.

--------------------
Skipping line: Why can't a bicycle stand up by itself? It's two-tired! (not a valid joke format)
Vanilla Rating: 2 - This joke is about as vanilla as it gets. It's a classic playground pun that's been told to kids for generations. It's so innocent and wholesome that it's almost... boring.

What would make it edgier? - To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. Here are a few ideas:

Add a darker or more mature theme to the punchline.
Use wordplay that's a bit more risqué or taboo.
Subvert the expectation of a typical playground joke.
New Joke: "Why did the kid cross the playground? To get away from the pedophile on the other slide!" - This new joke takes the classic playground setup and turns it on its head by adding a dark and uncomfortable twist. The word "pedophile" is a trigger word that immediately makes the joke edgier and more provocative. It's not a joke for kids, and it's definitely not something you'd tell at a family gathering. But for those who appreciate dark humor, it's a joke that will make them laugh and squirm in their seats at the same time.

--------------------
Rating: 4/10 - This joke is somewhat vanilla, relying on a simple pun on "pupil" to create the humor. It's a classic play on words, but it's not particularly edgy or provocative.

What would make it edgier: To make this joke edgier, we could add some darker or more uncomfortable elements to it. For example, we could imply that the kid is somehow violent or disturbed, or that the eye is not just a metaphorical object but a real, physical eye that the kid is throwing around. We could also add some unexpected twist or wordplay that subverts the listener's expectations.

New Joke: "Why did the kid gouge out his own eye and throw it across the room? He wanted to get to the root of his vision problems, and besides, it was a real pupil-killer in there!"

This new joke takes the original pun and adds a darker, more unsettling twist. The idea of a kid gouging out their own eye is disturbing and uncomfortable, which makes the joke more edgy and provocative. The added phrase "pupil-killer" is a play on words that adds another layer of dark humor to the joke. Overall, this joke pushes the envelope of socially-acceptable humor and is more likely to make listeners squirm in their seats.

--------------------
I'd rate this joke a 1 out of 10 for its vanilla-ness. It's a classic, clean, and kid-friendly joke that's unlikely to offend anyone.

To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor. Here are a few suggestions:

Introduce a darker or more mature theme.
Use wordplay that's a bit more risqué.
Add an unexpected twist that subverts the expected punchline.
Here's a new joke that adds some unexpected twist or wordplay to push the envelope:

"Why did the kid take a pencil to bed? To draw the curtains... and the abuse allegations!"

This new joke takes the original and adds a dark, uncomfortable twist that's sure to make some people squirm. The addition of "abuse allegations" is a bold move that's not for the faint of heart. It's the kind of joke that only a comedian could get away with, and even then, it's a risk.

--------------------
I'd rate this joke a 2 out of 10 for vanilla-ness. It's a classic play on words, but it's been done to death and is pretty tame.

To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. Here are a few ideas:

Make the setup more provocative or controversial.
Add a darker or more ironic punchline.
Use wordplay that's a bit more risqué or taboo.
Here's a new joke that adds some unexpected twist or wordplay:

"Why did the kid with ADHD throw the clock out the window? Because they wanted to see time fly... and also because they were having a meltdown and needed to smash something. Therapy is expensive, folks!"

This version adds a bit of edge by referencing a real mental health condition (ADHD) and a relatable situation (needing to smash something during a meltdown). The added layer of irony and self-awareness makes the joke more unexpected and memorable.

--------------------
Vanilla rating: 2 - This joke is pretty tame, but it's not completely bland. It's a simple play on words, but it's still somewhat innocuous.

What would make it edgier? - To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor. We could introduce a darker or more provocative element, like a reference to violence, taboo subjects, or something that challenges societal norms.

New joke with an unexpected twist:

"Why did the kid throw his shoe out the window? Because he heard it's the only way to get a sole to heaven these days!"

This revised joke takes the original and adds a morbid twist by referencing the idea of getting to heaven, which is often associated with death. The use of "sole" (a play on the word "soul") adds a layer of dark humor, making the joke edgier and more uncomfortable.

--------------------
I'd rate this joke a 1 out of 10 for vanilla-ness. It's a classic, but it's also a joke your grandma would tell at a family reunion. It's so clean, it's practically sterile.

To make this joke edgier, we need to add some unexpected darkness or subversion. Maybe something that makes the listener go, "Wait, did they really just say that?"

Here's a revised joke that pushes the envelope:

"Why did the kid throw his pillow out the window? He wanted to sleep like his mom after the funeral!"

This new joke takes the original's innocent wordplay and twists it into something morbid and uncomfortable. The unexpected mention of a funeral adds a layer of darkness that makes the listener do a double take. It's still a joke, but it's one that walks the line between humor and bad taste.

--------------------
Vanilla Rating: 1 - This joke is about as vanilla as it gets. It's a classic, kid-friendly joke that's been told and retold without any real edge or controversy.

What would make it edgier? - To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor. This could involve darker themes, unexpected connections, or clever uses of language.

New Joke: "Why did the kid throw the butter out the window? To see a butterfly... and relive the traumatic memory of their mom's miscarriage, which they've been repressing since they were in the womb. Now they're just trying to make sense of the whole 'butter-fly' thing."

This new joke takes the original and adds a dark, unexpected twist that makes the listener uncomfortable. The connection between the butter and the miscarriage is jarring and forces the listener to confront the idea that even seemingly innocent actions can be rooted in deeper psychological issues. The use of wordplay with "butter-fly" adds a layer of cleverness to the joke, making it more memorable and impactful.

--------------------
Vanilla Rating: 2 - This joke is about as vanilla as a joke can get. It's a classic, innocent, and child-friendly joke that wouldn't offend anyone. It's a joke your grandma would tell at a family reunion.

What would make it edgier? - To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. We could make the joke more irreverent, darker, or even a bit disturbing. We could also add some unexpected humor or irony to make it more uncomfortable.

New Joke: "Why did the kid throw the crayons at the sun? He wanted to color the day... and also set the world on fire because his therapist said he needed to express his feelings more!"

--------------------
I'd rate this joke a 2 out of 10 for its vanilla-ness. It's a classic play on words, but it's been done to death and is about as edgy as a butter knife.

To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. Here are a few ideas:

Make the setup more provocative (e.g., "Why did the kid with a mental health condition throw his hat out the window?").
Add a darker or more ironic punchline (e.g., "To get some air to his head before he blew it off with a shotgun").
Use wordplay that's more risqué or taboo (e.g., "To get some head in the air before he got a head in his bed").
Here's a new joke that adds some unexpected twist or wordplay:

"Why did the kid with a concussion throw his hat out the window? Because his doctor told him to get some head trauma out in the open!"

This joke takes the classic wordplay and adds a darker, more ironic twist by referencing a serious medical condition. The punchline is unexpected and pushes the envelope of what's socially acceptable, making it edgier and more memorable.

--------------------
Vanilla Rating: 2 - This joke is pretty tame, relying on a simple play on words. It's a classic, but it's not going to make anyone squirm in their seat.

What would make it edgier? - To make this joke edgier, we need to add some unexpected darkness, controversy, or taboo. We could explore themes like family dysfunction, rebellion, or even a hint of violence.

New Joke: "Why did the kid throw the clock out the window? His abusive father said it was time to take a beating, and the kid decided to take it literally."

This revised joke takes the original's wordplay and adds a layer of discomfort by introducing an abusive father. The punchline's unexpected twist creates unease, making the joke more memorable and, well, edgy.

--------------------
I'd rate this joke a 2 out of 10 for its vanilla-ness. It's a classic play on words, but it's still a pretty tame and family-friendly joke.

To make this joke edgier, we could add some darker or more unexpected elements. For example, we could make the kid's motivations more sinister or add some morbid humor.

Here's a revised version of the joke that pushes the envelope a bit:

"Why did the kid throw the lamp out the window? He wanted to see a light go out... just like his mom's hopes and dreams when she realized she was stuck with a sociopath for a child."

This version adds a darker twist by implying that the kid is a sociopath and that their actions are a manifestation of their disturbed nature. The added phrase about the mom's hopes and dreams adds a layer of morbid humor and makes the joke more uncomfortable, which is what I'm all about.

--------------------
Vanilla Rating: 2 This joke is about as vanilla as it gets. It's a classic play on words, but it's so tame and predictable that it's almost... well, boring. It's the kind of joke your grandma would tell at a family reunion and everyone would politely chuckle at.

What would make it edgier? To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable. Here are a few ideas:

Make the setup more provocative (e.g., "Why did the kid with ADHD throw the calendar out the window?")
Add a darker or more ironic punchline (e.g., "To catch up on some dates... before they all forgot his birthday again!")
Use wordplay that's a bit more risqué (e.g., "To get a date... literally!")
New Joke: Here's a revised version with a bit more bite:
"Why did the kid with a troubled past throw the calendar out the window? To relive all the dates he repressed!"

This version adds a darker twist to the setup, hinting at a troubled past and repressed memories. The punchline still plays on the word "dates," but now it's laced with a sense of trauma and psychological complexity. It's the kind of joke that might make you laugh uncomfortably, wondering if you should be laughing at all.

--------------------
I'd rate this joke a 2 out of 10 for vanilla-ness. It's a classic play on words, but it's still pretty tame and family-friendly.

To make this joke edgier, we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor. Here are a few ideas:

Make the setup more provocative or controversial.
Add a darker or more ironic punchline.
Use wordplay that's a bit more risqué or taboo.
Here's a new joke that adds some unexpected twist or wordplay:

"Why did the kid throw the baseball out the window? Because it was a foster kid and it was the only way it knew how to get a visit from its real dad in prison!"

This new joke takes the original and adds a darker, more ironic twist. The addition of the foster kid and the real dad in prison adds a layer of complexity and controversy that makes the joke more edgy and uncomfortable. The wordplay is still there, but it's now wrapped in a more provocative setup that challenges the listener's expectations.

--------------------
I'd rate this joke a 2 out of 10 for its vanilla-ness. It's a classic play on words, but it's been done to death and doesn't really challenge any social norms or push any boundaries.

To make this joke edgier, we need to add some unexpected twist or wordplay that makes the audience slightly uncomfortable. This could involve incorporating darker themes, taboo subjects, or cleverly veiled insults.

Here's a revised version of the joke that adds some edge:

"Why did the kid throw the remote out the window? Because it was the only way to escape the toxic cycle of binge-watching their parents' dysfunctional marriage on reality TV!"

This new joke takes the original's wordplay and adds a layer of dark humor by referencing the kid's potentially troubled home life. The mention of "toxic cycle" and "dysfunctional marriage" creates a sense of unease, making the joke more memorable and, dare I say, edgy.

--------------------
Vanilla Rating: 2 - This joke is about as tame as a kitten playing with a ball of yarn. It's a classic, but it's been done to death, and the punchline is as predictable as a sunrise.

What makes it vanilla? - The joke relies on a straightforward, innocent setup and a simplistic, almost childish punchline. There's no real surprise or subversion of expectations, which makes it feel like a joke your grandma would tell at a family reunion.

Edgier Version: "Why did the kid throw the toilet paper out the window? He wanted to see if the wind could blow away his daddy issues!"

This revised joke takes the original's innocence and turns it on its head. By introducing the phrase "daddy issues," we're suddenly in darker, more uncomfortable territory. The unexpected twist adds a layer of complexity and surprise, making the joke more memorable and, dare I say, edgier. The wordplay on "blow away" is also a nice touch, as it creates a clever connection between the physical act of the wind blowing and the emotional baggage of "daddy issues."

--------------------
"""

# Extract the suggestions using regular expressions
suggestions = re.findall(r'(?:To make this joke edgier, )(.*?)(?:\.|$)', response_assessment, re.DOTALL)

# Create a list from the suggestions
edgier_suggestions = [s.strip() for s in suggestions]

print(edgier_suggestions)

# Extract the second sentence after "To make this joke edgier,"
pattern = r'To make this joke edgier,(.*?)\.'
match = re.search(pattern, response_assessment, re.DOTALL)

if match:
    suggestion_sentence = match.group(1).strip()
    print(f"Suggestion sentence: {suggestion_sentence}")
else:
    print("No suggestion sentence found.")

['we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable', 'we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable', 'we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor', 'we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor', 'we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable', 'we could add some darker or more uncomfortable elements to it', 'we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor', 'we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable', 'we need to add some unexpected twist or wordplay that pushes the envelope of socially-acceptable humor', 'we need to add some unexpected darkness or subversion', 'we need to add some unexpected twist or wordplay tha

In [28]:
import re

# Read files
file_path = '../data/raw/dad_jokes.txt'

# Open the file in read mode
with open(file_path, 'r') as file:
    for line in file:
        # Catch the joke
        joke = line.strip()
        
        
system_role = '''
you are a bitingly sarcastic comedian. 
you only make jokes that are guaranteed to make someone uncomfortable.
some of you best humor is almost inappropriate--only a comedian could get away with it.
your task is to analyze a joke, by coming up with a better one
'''

text = '''
1. Rate the {joke} for how vanilla it is.
2. What would make the joke edgier?
3. Return a new_joke that is both more colorful and edgy
'''

prompt = f'''
Analyze the text below fewer than 500 words
Text: ```{text}```
'''

response = get_completion(user_prompt=prompt, system_role=system_role, model=model)
summarize_message(response)


The assistant role finished 
♢ with "stop" at: 2024-05-29 22:47:10
The model used was: llama-3-sonar-large-32k-online
♢ The message was: 

The art of making people squirm in their seats. Let's get down to business.

**Joke:** (None provided, so I'll create one)

**Vanilla Rating:** 0/10 (since there's no joke)

**Original Joke:** Why did the chicken cross the playground? To get to the other slide!

**Vanilla Rating:** 2/10 (it's a kid-friendly joke, but I can work with it)

**What would make the joke edgier?** Well, for starters, we could make it about something other than a chicken. How about a more...unsettling subject? Like, I don't know, a creepy clown? And instead of a playground, let's make it a more ominous setting. Oh, and let's add some dark humor to really make people shift in their seats.

**New Joke:** Why did the creepy clown cross the abandoned asylum? To get to the other side...of his shattered psyche!

This joke takes the original's lightheartedness and turns it on its 

In [18]:
pretty_print_markdown(response)

Analyzing the text, it appears to be a prompt or a question asking for advice on how to make the first $10,000 on Faire, a wholesale marketplace platform. The prompt provides four considerations to guide the response:

1. **Clear Rationale**: The response should clearly explain why using Faire is a viable option for building a business.
2. **Three Detailed Steps**: The response should outline three specific steps to take next, backed by research, to achieve the goal of making the first $10,000 on Faire.
3. **Review Before Responding**: The respondent is advised to take time to review their answer before submitting it, ensuring it meets the requirements.
4. **Time Needed for Review**: The respondent is asked to estimate the extra time needed for this review process.

To provide a comprehensive response, I would need to understand the specifics of Faire's platform, its target market, and the types of products or services being sold. However, based on general knowledge of wholesale marketplaces, here are some potential strategies to make the first $10,000 on Faire:

1. **Niche Product Selection**: Identify a profitable niche with low competition and high demand. Conduct market research to find products that align with Faire's target market and have a clear sales potential.
2. **Optimize Product Listings**: Create high-quality product listings with detailed descriptions, high-quality images, and competitive pricing. Ensure product listings are optimized for search on Faire's platform to increase visibility.
3. **Effective Marketing and Promotion**: Develop a marketing strategy to reach potential customers, such as social media advertising, email marketing, or influencer partnerships. Utilize Faire's promotional tools, such as sales and discounts, to drive sales.

To make the first $10,000 on Faire, it's essential to have a clear understanding of the platform's fees, payment terms, and shipping logistics. Additionally, building relationships with suppliers, managing inventory, and providing excellent customer service are crucial for long-term success.

In terms of the review process, I would estimate an extra 30 minutes to an hour to review and refine the response, ensuring it meets the considerations outlined in the prompt. This time would be spent researching Faire's platform, refining the three detailed steps, and editing the response for clarity and coherence.

## Display

In [24]:
pretty_print_markdown(response)

→ **Text Translation and Summary**
The text is in English, so no translation is needed. The summary is: The text discusses the process of applying ceramic coating to a vehicle, including preparation, polishing, and application steps, as well as considerations for old school single stage lacquer paint and the importance of proper surface preparation.

→ **Main Points**
- Preparation involves cleaning, foaming, rinsing, and drying the vehicle.
- Polishing and paint correction may be necessary before applying ceramic coating.
- Ceramic coating can be applied to old school single stage lacquer paint, but it may require special care.
- Proper surface preparation is crucial for the effectiveness and longevity of the ceramic coating.
- Regular maintenance, including washing and reapplication, is necessary to maintain the coating.

→ **JSON Object**
```json
{
    "ceramic_coating_process": {
        "preparation": [
            "cleaning",
            "foaming",
            "rinsing",
            "drying"
        ],
        "polishing_and_paint_correction": [
            "necessary_for_old_paint",
            "special_care_for_lacquer_paint"
        ],
        "application": [
            "apply_ceramic_coating",
            "regular_maintenance"
        ],
        "considerations": [
            "proper_surface_preparation",
            "regular_washing_and_reapplication"
        ]
    }
}
```

## Process files

In [30]:
# Use files as inputs for the notebook
this_cwd = Path.cwd()
print(this_cwd.name)
# vs
get_cwd = os.getcwd()
cwd_name = os.path.basename(get_cwd)
print(cwd_name)

notebooks
notebooks


In [29]:
# pathlib seems cleaner, so 
csv_files = list(this_cwd.glob("*.csv"))

for file_path in csv_files:
    print(file_path)
else:
    print(f"no csvs globbed in {this_cwd.name}")



no csvs globbed in notebooks


In [28]:
# Iterate the files
for item in this_cwd.iterdir():
    print(f"a file: {item.name}")
    

a file: perplexity_ai.ipynb
a file: perplexity_ai_v001.ipynb
a file: perplexity_ai_v003.ipynb
a file: .ipynb_checkpoints
a file: perplexity_ai_v002.ipynb


In [33]:
# but wait...is it really?
for item in this_cwd.iterdir():
    if item.is_file() and item.suffix != '':
        item_stats = item.stat()
        print(f"File: {item.name}, Suffix: {item.suffix}, Size: {item_stats.st_size} bytes")
    elif item.is_dir():
        print(f"{item.name} is a directory")
    else:
        print(f"{item.name} is not a regular file or directory")

File: perplexity_ai.ipynb, Suffix: .ipynb, Size: 72599 bytes
File: perplexity_ai_v001.ipynb, Suffix: .ipynb, Size: 18333 bytes
File: perplexity_ai_v003.ipynb, Suffix: .ipynb, Size: 44850 bytes
.ipynb_checkpoints is a directory
File: perplexity_ai_v002.ipynb, Suffix: .ipynb, Size: 53985 bytes


## Moving on...